CG
Input: $x_0$, $r_0 = Ax_0-b$, $p_0 = -r_0$

For k = 0,1,2
    $\alpha_k = -\frac{r_k^Tp_k}{p_k^TAp_k}$
    $x_{k+1} = x_k +\alpha_k p_k$
    $r_{k+1} = r_k +\alpha_k A p_k$
    $\beta_{k+1} = \frac{r_{k+1}Ap_k}{p_kAp_k}$
    $p_{k+1}= -r_{k+1} + \beta_{k+1}p_{k}$

In [1]:
import numpy as np 
import numpy as np
import scipy.sparse as sp
import matplotlib.pyplot as plt






def conj_grad(A,b,x0,verbose=True):
    r0 = A@x0 -b
    p0 = -r0
    r = r0
    p = p0
    x=x0
    eps = 1e-12
    if verbose:
        res = []
        res.append(np.linalg.norm(r))
    k = 1
    while np.linalg.norm(r)>eps:
        alpha = (r.T@r)/(p.T@A@p)
        x =x+ alpha*p
        rold = r
        r = r+ alpha*A@p
        if verbose:
            res.append(np.linalg.norm(r))
        beta = (r.T@r)/(rold.T@rold)
        p = -r + beta*p

    if verbose:
        return x,res

    return x

def conj_grad_prec(A,b,x0,M=None,verbose=False):
    r0 = A@x0 -b
    y0 = sp.linalg.spsolve(M,r0)

    p0 = -y0
    r = r0
    p = p0
    x=x0
    y=y0
    eps = 1e-12
    if verbose:
        res = []
        res.append(np.linalg.norm(r))
    while np.linalg.norm(r)>eps:
        # alpha = -(r.T@p)/(p.T@A@p)
        alpha = (r.T@y)/(p.T@A@p)
        x =x+ alpha*p
        rold = r
        yold = y
        r = r+ alpha*A@p
        if verbose:
            res.append(np.linalg.norm(r))
        y = sp.linalg.spsolve(M,r)
        beta = (r.T@y)/(rold.T@yold)
        p = -y + beta*p
    if verbose:
        return x,res

    return x

In [2]:
# import numpy as np
# import scipy.sparse as sp
# import matplotlib.pyplot as plt

# def random_walk_in_maze():
#     # Load data
#     data_down = np.loadtxt("maze_data_down.csv", delimiter=",", skiprows=1)[:, 1:]
#     data_right = np.loadtxt("maze_data_right.csv", delimiter=",", skiprows=1)[:, 1:]
    
#     # Draw the maze
#     fig = 1
#     draw_maze(data_down, data_right, fig)
    
#     # Each maze cell is a node
#     m, n = data_down.shape
#     N = m * n
#     A = make_adjacency_matrix(data_down, data_right)
    
#     # Convert adjacency matrix to stochastic matrix
#     row_sums = A.sum(axis=1).A.flatten()
#     R = sp.diags(1.0 / row_sums)
#     P = R @ A
    
#     # Setup linear system for the committor problem
#     exitA = 0  # Python uses 0-based indexing
#     exitB = N - 1
#     x = np.zeros(N)
#     x[exitB] = 1
#     I = sp.eye(N)
#     L = P - I
#     b = -L @ x
#     ind_exits = {exitA, exitB}
#     ind_unknown = list(set(range(N)) - ind_exits)
    
#     # Make the system symmetric positive definite
#     r_sqrt = np.sqrt(row_sums)
#     D = sp.diags(r_sqrt)
#     Dinv = sp.diags(1.0 / r_sqrt).tocsr()  # Convert Dinv to CSR format
#     Lsymm = D @ L @ Dinv
#     bsymm = D @ b
    
#     # Solve the linear system
#     x[ind_unknown] = sp.linalg.spsolve(Lsymm[ind_unknown][:, ind_unknown], bsymm[ind_unknown])
#     solution = x[ind_unknown].copy()
#     x[ind_unknown] = Dinv[ind_unknown, :][:, ind_unknown] @ x[ind_unknown]
    
#     # Visualize the solution
#     plt.figure(3)
#     committor = x.reshape(m, n)
#     plt.imshow(committor, cmap='viridis')
#     draw_maze(data_down, data_right, 3)
    
#     # Find eigenvalues of Lsymm
#     evals = np.sort(np.real(sp.linalg.eigs(-Lsymm, k=min(N-1, 10), return_eigenvectors=False)))
#     plt.figure(4)
#     plt.plot(evals, '.', markersize=15)
#     plt.grid(True)
#     plt.gca().set_xlabel('eigenvalue index', fontsize=20)
#     plt.gca().set_ylabel('eigenvalue', fontsize=20)
#     plt.gca().tick_params(axis='both', which='major', labelsize=20)
    
#     # Save the solution
#     A_symm = -Lsymm[ind_unknown][:, ind_unknown]
#     b_symm = -bsymm[ind_unknown]
#     np.savez("maze_linear_system.npz", A=A_symm, b=b_symm, solution=solution)

# def draw_maze(data_down, data_right, fig):
#     m, n = data_down.shape
#     plt.figure(fig)
#     plt.clf()
#     line_width = 3
#     col = 'k'
    
#     # Plot outer lines
#     plt.plot(0.5 + np.arange(1, n+1), 0.5 + np.zeros(n), color=col, linewidth=line_width)
#     plt.plot(0.5 + np.arange(n), 0.5 + m * np.ones(n), color=col, linewidth=line_width)
#     plt.plot(0.5 + np.zeros(m), 0.5 + np.arange(1, m+1), color=col, linewidth=line_width)
#     plt.plot(0.5 + n * np.ones(m), 0.5 + np.arange(m), color=col, linewidth=line_width)
    
#     # Plot vertical lines
#     for i in range(m):
#         for j in range(n-1):
#             if data_right[i, j] == 0:
#                 plt.plot([0.5 + j, 0.5 + j], [0.5 + i, 0.5 + i + 1], color=col, linewidth=line_width)
    
#     # Plot horizontal lines
#     for j in range(n):
#         for i in range(m-1):
#             if data_down[i, j] == 0:
#                 plt.plot([0.5 + j, 0.5 + j + 1], [0.5 + i, 0.5 + i], color=col, linewidth=line_width)
    
#     plt.axis('image')
#     plt.axis('off')

# def make_adjacency_matrix(data_down, data_right):
#     m, n = data_down.shape
#     mn = m * n
#     A = sp.lil_matrix((mn, mn))
    
#     for i in range(m):
#         for j in range(n-1):
#             if data_right[i, j] == 1:
#                 ind = j * m + i
#                 A[ind, ind + m] = 1
#                 A[ind + m, ind] = 1
    
#     for j in range(n):
#         for i in range(m-1):
#             if data_down[i, j] == 1:
#                 ind = j * m + i
#                 A[ind, ind + 1] = 1
#                 A[ind + 1, ind] = 1
                
#     return A.tocsr()

# if __name__ == "__main__":
#     random_walk_in_maze()


In [3]:
# import numpy as np
# import scipy.sparse as sp
# import matplotlib.pyplot as plt
# from scipy.sparse.linalg import spsolve

# def random_walk_in_maze():
#     # Load data
#     data_down = np.loadtxt("maze_data_down.csv", delimiter=',', skiprows=1)[:, 1:]
#     data_right = np.loadtxt("maze_data_right.csv", delimiter=',', skiprows=1)[:, 1:]

#     # Draw maze
#     fig = 1
#     draw_maze(data_down, data_right, fig)

#     # Generate adjacency matrix
#     m, n = data_down.shape
#     N = m * n
#     A = make_adjacency_matrix(data_down, data_right)

#     # Convert adjacency matrix to stochastic matrix
#     row_sums = A.sum(axis=1).A.flatten()
#     R = sp.diags(1.0 / row_sums)
#     P = R @ A

#     # Setup linear system for the committor problem
#     exitA = 0
#     exitB = N - 1
#     x = np.zeros(N)
#     x[exitB] = 1
#     I = sp.eye(N)
#     L = P - I
#     b = -L @ x
#     ind_exits = {exitA, exitB}
#     ind_unknown = list(set(range(N)) - ind_exits)

#     # Make the system symmetric positive definite
#     r_sqrt = np.sqrt(row_sums)
#     D = sp.diags(r_sqrt)
#     Dinv = sp.diags(1.0 / r_sqrt)
#     Lsymm = D @ L @ Dinv
#     bsymm = D @ b

#     # Solve the linear system
#     # Convert Lsymm and Dinv to CSR format for subscriptable access
#     Lsymm_csr = Lsymm.tocsr()
#     Dinv_csr = Dinv.tocsr()

#     # Solve for the unknown indices
#     x[ind_unknown] = spsolve(Lsymm_csr[ind_unknown, :][:, ind_unknown], bsymm[ind_unknown])
#     solution = x[ind_unknown]
#     x[ind_unknown] = Dinv_csr[ind_unknown, :][:, ind_unknown] @ x[ind_unknown]

#     # Visualize the solution
#     plt.figure(3)
#     committor = x.reshape((m, n))
#     plt.imshow(committor, cmap='viridis', interpolation='none')
#     draw_maze(data_down, data_right, 3)

#     # Find eigenvalues of Lsymm
#     evals = np.sort(np.real(np.linalg.eigvals(-Lsymm.toarray())))[::-1]
#     plt.figure(4)
#     plt.plot(evals, '.', markersize=15)
#     plt.grid(True)
#     plt.gca().tick_params(labelsize=20)
#     plt.xlabel('eigenvalue index', fontsize=20)
#     plt.ylabel('eigenvalue', fontsize=20)

#     # Save the solution
#     A = -Lsymm_csr[ind_unknown, :][:, ind_unknown]
#     b = -bsymm[ind_unknown]
#     np.savez("maze_linear_system.npz", A=A.toarray(), b=b, solution=solution)


# def draw_maze(data_down, data_right, fig):
#     m, n = data_down.shape
#     plt.figure(fig)
#     plt.clf()
#     line_width = 3
#     col = 'k'
#     # Plot outer lines
#     plt.plot(0.5 + np.arange(1, n+1), 0.5 + np.zeros(n), color=col, linewidth=line_width)
#     plt.plot(0.5 + np.arange(n), 0.5 + m * np.ones(n), color=col, linewidth=line_width)
#     plt.plot(0.5 + np.zeros(m), 0.5 + np.arange(1, m+1), color=col, linewidth=line_width)
#     plt.plot(0.5 + m * np.ones(m), 0.5 + np.arange(m), color=col, linewidth=line_width)
    
#     # Plot vertical lines
#     for i in range(m):
#         for j in range(n - 1):
#             if data_right[i, j] == 0:
#                 plt.plot([0.5 + j, 0.5 + j], [0.5 + i - 1, 0.5 + i], color=col, linewidth=line_width)

#     # Plot horizontal lines
#     for j in range(n):
#         for i in range(m - 1):
#             if data_down[i, j] == 0:
#                 plt.plot([0.5 + j - 1, 0.5 + j], [0.5 + i, 0.5 + i], color=col, linewidth=line_width)

#     plt.gca().invert_yaxis()
#     plt.axis('off')
#     plt.gca().set_aspect('equal')


# def make_adjacency_matrix(data_down, data_right):
#     m, n = data_down.shape
#     mn = m * n
#     A = sp.lil_matrix((mn, mn))

#     for i in range(m):
#         for j in range(n - 1):
#             if data_right[i, j] == 1:
#                 ind = j * m + i
#                 A[ind, ind + m] = 1
#                 A[ind + m, ind] = 1

#     for j in range(n):
#         for i in range(m - 1):
#             if data_down[i, j] == 1:
#                 ind = j * m + i
#                 A[ind, ind + 1] = 1
#                 A[ind + 1, ind] = 1

#     return A.tocsr()

# if __name__ == "__main__":
#     random_walk_in_maze()

In [4]:
import numpy as np

# Load the data
solution = np.loadtxt('solution.csv', delimiter=',')
A = np.loadtxt('A_matrix.csv', delimiter=',')
b = np.loadtxt('b_vector.csv', delimiter=',')
A = sp.csc_matrix(A)

# Display the data
print('Solution:', solution)
print('A Matrix:', A)
print('b Vector:', b)

Solution: [0.         0.00632911 0.01265823 0.01898734 0.02531646 0.05696203
 0.06329114 0.06329114 0.06329114 0.06329114 0.06329114 0.06329114
 0.06329114 0.06329114 0.06329114 0.06329114 0.06329114 0.06329114
 0.06329114 0.06329114 0.         0.         0.31012658 0.30379747
 0.03164557 0.05063291 0.06962025 0.06329114 0.06329114 0.06329114
 0.06329114 0.06329114 0.06329114 0.06329114 0.06329114 0.06329114
 0.06329114 0.06329114 0.06329114 0.06329114 0.         0.
 0.3164557  0.29746835 0.03797468 0.0443038  0.07594937 0.06329114
 0.06329114 0.06329114 0.06329114 0.06329114 0.06329114 0.06329114
 0.06329114 0.06329114 0.06329114 0.06329114 0.06329114 0.06329114
 0.         0.         0.32278481 0.29113924 0.28481013 0.0443038
 0.08227848 0.06329114 0.06329114 0.06329114 0.06329114 0.06329114
 0.06329114 0.06329114 0.06329114 0.06329114 0.06329114 0.06329114
 0.06329114 0.06329114 0.         0.         0.32911392 0.26582278
 0.27848101 0.0443038  0.08860759 0.09493671 0.06329114 0.120

In [5]:
cg_solution,cg_residuals = conj_grad(A,b,np.zeros(b.shape),verbose=True)

In [6]:
# def ichol(A):
#     A = sp.csc_matrix(A)
#     ilu = sp.linalg.spilu(A, drop_tol=1e-12, fill_factor=4.0)
#     return ilu.L



# C = ichol(A)
# M = C@C.T
C = np.loadtxt('prec.csv', delimiter=',')
M=C@C.T
M=sp.csc_matrix(M)

cgp_solution,cgp_residuals = conj_grad_prec(A,b,np.zeros(b.shape),M=M,verbose=True)

In [14]:
plt.figure(figsize=(10,5))
plt.plot(cg_residuals)
plt.plot(cgp_residuals)

plt.yscale('log')
plt.legend(['CG','CGP'])
plt.ylabel('Residual')
plt.xlabel('Iteration')
plt.title('CG vs CGP Residual')
plt.show()

TypeError: Value after * must be an iterable, not int